In [ ]:
# 加载数据、第一层配对计算 latency、画时序图（无需 analyze_wrr_results，可单独拷贝到 Win 运行）
import re
from pathlib import Path
import matplotlib.pyplot as plt

MAX_LATENCY_MS = 120000

def parse_receiver_log(f, marker="P4 is cool"):
    ts, pending = [], None
    for line in open(f, 'r'):
        if 'got a packet' in line: pending = None; continue
        m = re.search(r'received at time : ([\d.]+)', line)
        if m: pending = float(m.group(1)); continue
        if marker in line and pending is not None: ts.append(pending); pending = None
    return ts

def parse_sender_log(f):
    out = []
    for line in open(f, 'r'):
        m = re.search(r'sent (\d+) packets until now : ([\d.]+)', line)
        if m: out.append((int(m.group(1)), float(m.group(2))))
    return out

outputs_dir = Path("./outputs/0131/test1")  # 修改为你的 outputs 路径（含 receiver_h_r*.txt, sender_h*.txt）
data = []
for fid in range(3):
    recv = parse_receiver_log(outputs_dir / f"receiver_h_r{fid+1}.txt")
    send = parse_sender_log(outputs_dir / f"sender_h{fid+1}.txt")
    for i, rt in enumerate(recv):
        if i >= len(send): break
        _, st = send[i]
        lat = (rt - st) * 1000
        if 0 <= lat < MAX_LATENCY_MS:
            data.append((rt, fid, lat))
start_t, end_t = min(r for r,_,_ in data), max(r for r,_,_ in data)
print(f"Matched: {len(data)} | Time: {start_t:.2f} - {end_t:.2f} s")

In [ ]:
# 时序图：可设置 zoom_start, zoom_end 放大某段时间；use_relative_time=True 使 x 轴为“自首个包起的秒数”，便于不同运行对比
zoom_start, zoom_end = 0, 30   # None 表示用全范围；如 0, 30 表示放大前 30 秒
use_relative_time = True       # True: x 轴 0 为首个包时刻，便于不同 quantum 运行对比

xmin = start_t + zoom_start if zoom_start is not None else start_t
xmax = start_t + zoom_end if zoom_end is not None else end_t
data_zoom = [(r,f,l) for r,f,l in data if xmin <= r <= xmax]
x_offset = start_t if use_relative_time else 0   # 相对时间时减去 start_t

fig, ax = plt.subplots(figsize=(16, 5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
for fid in range(3):
    pts = [(r - x_offset, fid+1) for r, f, _ in data_zoom if f == fid]
    if pts: ax.scatter([p[0] for p in pts], [p[1] for p in pts], c=colors[fid], s=12, alpha=0.8, label=f'Flow {fid}')
ax.set_xlim(xmin - x_offset, xmax - x_offset); ax.set_ylim(0.5, 3.5)
ax.set_yticks([1,2,3]); ax.set_yticklabels(['Flow 0', 'Flow 1', 'Flow 2'], fontsize=12)
ax.tick_params(axis='both', labelsize=12)
ax.xaxis.get_offset_text().set_fontsize(12)   # 右下角科学计数偏移文字的字体
ax.set_xlabel('Time since first packet (s)' if use_relative_time else 'Time (s)', fontsize=12)
ax.set_ylabel('Flow', fontsize=12)
ax.legend(loc='upper right', ncol=3, fontsize=12)
plt.tight_layout(); plt.show()

In [ ]:
# 整个实验过程时序图（全时间范围）；use_relative_time=True 使不同实验对齐
use_relative_time_full = True
x_offset_full = start_t if use_relative_time_full else 0

fig, ax = plt.subplots(figsize=(16, 5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
for fid in range(3):
    pts = [(r - x_offset_full, fid+1) for r, f, _ in data if f == fid]
    if pts: ax.scatter([p[0] for p in pts], [p[1] for p in pts], c=colors[fid], s=6, alpha=0.7, label=f'Flow {fid}')
ax.set_xlim(0 if use_relative_time_full else start_t, end_t - x_offset_full)
ax.set_ylim(0.5, 3.5)
ax.set_yticks([1,2,3]); ax.set_yticklabels(['Flow 0', 'Flow 1', 'Flow 2'], fontsize=12)
ax.tick_params(axis='both', labelsize=12)
ax.xaxis.get_offset_text().set_fontsize(12)
ax.set_xlabel('Time since first packet (s)' if use_relative_time_full else 'Time (s)', fontsize=12)
ax.set_ylabel('Flow', fontsize=12)
ax.legend(loc='upper right', ncol=3, fontsize=12)
plt.tight_layout(); plt.show()

In [ ]:
# seq_num 匹配：适配 log_every > 1（如 31 samples per flow），用 packet_seq 查找 sender
data_sampled = []
for fid in range(3):
    recv = parse_receiver_log(outputs_dir / f"receiver_h_r{fid+1}.txt")
    send = parse_sender_log(outputs_dir / f"sender_h{fid+1}.txt")
    send_time_by_seq = {seq: t for seq, t in send}
    for recv_idx, rt in enumerate(recv):
        packet_seq = recv_idx + 1
        if packet_seq not in send_time_by_seq:
            continue
        st = send_time_by_seq[packet_seq]
        lat = (rt - st) * 1000
        if 0 <= lat < MAX_LATENCY_MS:
            data_sampled.append((rt, fid, lat, packet_seq))
if data_sampled:
    start_s, end_s = min(r for r,_,_,_ in data_sampled), max(r for r,_,_,_ in data_sampled)
    print(f"Matched (seq_num): {len(data_sampled)} samples | Time: {start_s:.2f} - {end_s:.2f} s")
    for fid in range(3):
        n = sum(1 for _,f,_,_ in data_sampled if f == fid)
        print(f"  Flow {fid}: {n} samples")

In [ ]:
# 每个 flow 的 31 samples 时序图：x = 相对时间，y = latency (ms)
if not data_sampled:
    print("No sampled data (run previous cell first)")
else:
    x_offset_s = start_s
    fig, ax = plt.subplots(figsize=(14, 6))
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    for fid in range(3):
        pts = [(r - x_offset_s, lat) for r, f, lat, _ in data_sampled if f == fid]
        if pts:
            xs, ys = [p[0] for p in pts], [p[1] for p in pts]
            ax.scatter(xs, ys, c=colors[fid], s=40, alpha=0.8, label=f'Flow {fid} ({len(pts)} samples)')
    ax.set_xlabel('Time since first packet (s)', fontsize=12)
    ax.set_ylabel('Latency (ms)', fontsize=12)
    ax.legend(loc='upper left', fontsize=12)
    ax.tick_params(axis='both', labelsize=12)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# 每个 flow 单独子图：展示 31 samples 的 latency 随时间变化
if data_sampled:
    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    for fid, ax in enumerate(axes):
        pts = [(r - start_s, lat) for r, f, lat, _ in data_sampled if f == fid]
        if pts:
            xs, ys = [p[0] for p in pts], [p[1] for p in pts]
            ax.scatter(xs, ys, c=colors[fid], s=50, alpha=0.8)
        ax.set_ylabel(f'Flow {fid}\nLatency (ms)', fontsize=11)
        ax.grid(True, alpha=0.3)
    axes[-1].set_xlabel('Time since first packet (s)', fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
# Latency CDF 图：每个 flow 的 31 samples 累积分布
if data_sampled:
    import numpy as np
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    for fid in range(3):
        lats = sorted([lat for _, f, lat, _ in data_sampled if f == fid])
        if lats:
            n = len(lats)
            x = sorted(lats)
            y = np.arange(1, n + 1) / n
            ax.plot(x, y, c=colors[fid], linewidth=2, label=f'Flow {fid} (n={n})')
    ax.set_xlabel('Latency (ms)', fontsize=12)
    ax.set_ylabel('CDF (Cumulative Fraction)', fontsize=12)
    ax.legend(loc='lower right', fontsize=12)
    ax.tick_params(axis='both', labelsize=12)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
# 打印所有 sample 的 latency 值
if data_sampled:
    for fid in range(3):
        samples = [(pkt, lat) for _, f, lat, pkt in data_sampled if f == fid]
        samples.sort(key=lambda x: x[0])
        print(f"Flow {fid} ({len(samples)} samples):")
        for pkt, lat in samples:
            print(f"  packet {pkt}: {lat:.2f} ms")
        print()